https://github.com/langchain-ai/langgraph/blob/main/examples/rewoo/rewoo.ipynb?ref=blog.langchain.dev

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_KEY"]
search_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
search_key = os.environ["AZURE_SEARCH_ADMIN_KEY"]

In [2]:
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI

from langchain.vectorstores.azuresearch import AzureSearch
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
)

import re

## [00] define a state dict to contain - task, plan, steps, and other variables.

In [3]:
from typing import TypedDict, List


class ReWOO(TypedDict):
    task: str
    plan_string: str
    steps: List
    results: dict
    result: str

## [0] Embedding/vectorisation

In [4]:
# For RAG

embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version="2023-09-01-preview",
    chunk_size=1 
)
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=search_endpoint,
    azure_search_key=search_key,
    index_name="boardai03",
    embedding_function=embeddings.embed_query,
)

retriever = vector_store.as_retriever(search_key="hybrid", search_kwargs={"k": 2})

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

[llm]

In [5]:
llm = AzureChatOpenAI(
    deployment_name="gpt-4",
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version="2023-09-01-preview",    
)

## [1] Planner
https://github.com/langchain-ai/langgraph/blob/main/examples/rewoo/rewoo.ipynb?ref=blog.langchain.dev

In [37]:
prompt = """For the following task, make plans that can solve the problem step by step. For each plan, indicate \
which external tool together with tool input to retrieve evidence. You can store the evidence into a \
variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:

(1) RAG_LLM[input]: A pretrained RAG+LLM like yourself. Useful when you need to act with general
world knowledge and common sense + specific knowledge and data especially for Board and secretary role. Prioritize it when you are confident in solving the problem
yourself. Input can be any instruction.

(2) EMAIL[input]: Worker that send emails to board members to ask write board paper, check status of the return of those email and chase. 
Input will be a list of board members and agendas.

(3) COMPILE[input]:Worker that add all the separate paper into one board paper. input will be multiple board paper pieces from board members.

(4) CIRCULATE[input]: Worker that send compiled board paper to board members and get feed back and fix. Input will be a board paper. 

For example,
Task: execute this: To create a board paper, first, based on an approved agenda, the secretary sends an email to the NEC (or board members). The NEC then writes the paper. 
After the paper is written, the secretary compile, circulates and distributes the paper to the board members.


Plan: Find out what is approved agenda. #E1 = RAG+LLM[what is approved agenda]

Plan: Find out who are the NEC or board members. #E2 = RAG+LLM[who are the NEC or board members]

Plan: the secretary sends an email to the NEC (or board members).The NEC writes the paper. #E3 = sendEmail_chase_gatherPaper[#E2]

Plan: compilePaper #E4 = compilePaper[#E3] 

Plan: After the paper is written, the secretary circulates and distributes the paper to the board members. #E5 = circulatePaper[#E4]


Begin! 
Describe your plans with rich details. Each Plan should be followed by only one #E.


\nTask: {task}
\nContext: {context}


\nAnswer:""" 


PRB HERE
\nQuestion: {question}
\nContext: {context}

In [7]:
# rag_prompt = hub.pull("rlm/rag-prompt")
# rag_prompt

In [38]:
# simply Converting prompt into p_prompt: ChatPromptTemplate as langchain receives in that form

# p_prompt = ChatPromptTemplate(input_variables=['context', 'question'], 
#                             metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, 
#                             messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template=prompt))])


# same result as code above
p_prompt = ChatPromptTemplate.from_messages([("user", prompt)])

Instead of model(llm+rag), test llm only

In [9]:
# # Use LLM + RAG

# model = (
# {"context": retriever | format_docs, "task": RunnablePassthrough()}
# | p_prompt
# | llm
# | StrOutputParser()
# )

# query = "how do I create a board paper?"
# res = model.invoke(query)
# print(res)
# type(res) #str

## PRB HERE

In [35]:
## TESTING LLM
# task = "how do I create a board paper?"
# res = llm.invoke(task)
# print(res)
# type(res) #langchain_core.messages.ai.AIMessage



content='Creating a board paper involves several steps:\n\n1. Understand The Purpose: The first step is to understand the purpose of the board paper. Why is it required? Is it to inform board members about a particular issue? Or is it to seek approval for an initiative?\n\n2. Research: Once you understand the purpose, the next step is to gather all the necessary information. This may involve conducting research, talking to relevant staff members, or reviewing documents and reports.\n\n3. Structure the Paper: A typical board paper has a clear structure: \n\n   a. Title: It should be concise and clearly indicate the subject of the paper.\n\n   b. Executive Summary: This provides an overview of the issue at hand, the decision required, and a summary of the main points raised in the paper.\n\n   c. Background: This section provides relevant information and context about the issue.\n\n   d. Discussion: This is the main body of the paper where you present the details of the issue, the analys

langchain_core.messages.ai.AIMessage

In [31]:
task = "how do I create a board paper?"

In [32]:
model = llm

Planner Node

To connect the planner to our graph,
we will create a "get_plan" node 
- accepts the ReWOO state and 
- returns with a state update for the steps and plan_string fields.

### get_plan - call this from langgraph later 

PRB HERE



In [42]:
# Regex to match expressions of the form  "Plan: E#... = ...[...]"
regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]" 

prompt_template = ChatPromptTemplate.from_messages([("user", prompt)])
planner = prompt_template | model


def get_plan(state: ReWOO):

    task = state["task"]
    context = "empty for now"

    result = planner.invoke({"task": task, "context": context}) #ask agent about the each task/plan again 
    
    # Find all matches in the sample text    
    matches = re.findall(regex_pattern, result.content) 
    # result.content = plan_string e.g. Plan: Find out what is approved agenda. #E1 = RAG+LLM[what is approved agenda]
    # matches = a list =  [Step description, step name(#E1, #E2, ...), tool, input]

    return {"steps": matches, "plan_string": result.content}


In [41]:
prompt_template

ChatPromptTemplate(input_variables=['context', 'task'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'task'], template='For the following task, make plans that can solve the problem step by step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)\n\nTools can be one of the following:\n\n(1) RAG_LLM[input]: A pretrained RAG+LLM like yourself. Useful when you need to act with general\nworld knowledge and common sense + specific knowledge and data especially for Board and secretary role. Prioritize it when you are confident in solving the problem\nyourself. Input can be any instruction.\n\n(2) EMAIL[input]: Worker that send emails to board members to ask write board paper, check status of the return of those email and chase. \nInput will be a list of board members and agendas.\n\n(3) COMPILE[inp

In [40]:
planner

ChatPromptTemplate(input_variables=['context', 'task'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'task'], template='For the following task, make plans that can solve the problem step by step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)\n\nTools can be one of the following:\n\n(1) RAG_LLM[input]: A pretrained RAG+LLM like yourself. Useful when you need to act with general\nworld knowledge and common sense + specific knowledge and data especially for Board and secretary role. Prioritize it when you are confident in solving the problem\nyourself. Input can be any instruction.\n\n(2) EMAIL[input]: Worker that send emails to board members to ask write board paper, check status of the return of those email and chase. \nInput will be a list of board members and agendas.\n\n(3) COMPILE[inp

## [2] EXECUTOR

Define the tool execution node

In [22]:
# load plugins & test

from plugins.basic import BasicPlugins

plugin_instance = BasicPlugins()

result = plugin_instance.email_function()
result = plugin_instance.compile_paper()
result = plugin_instance.circulate_paper()


result

'circulate paper'

In [43]:
def _get_current_task(state: ReWOO):
    if state["results"] is None:
        return 1
    if len(state["results"]) == len(state["steps"]):
        return None
    else:
        return len(state["results"]) + 1
    

# initialise plugins
from plugins.basic import BasicPlugins
plugin_instance = BasicPlugins()


# do i need to mention all the tools here? semantic kernel doesn't need this stage...hrmmmmm try langchain ver first for now
    

def tool_execution(state: ReWOO):
    """Worker node that executes the tools of a given plan."""
    _step = _get_current_task(state)
    _, step_name, tool, tool_input = state["steps"][_step - 1] # state["steps"] = List   # how come this is a tuple like structure
    _results = state["results"] or {}

    for k, v in _results.items():
        tool_input = tool_input.replace(k, v)
    
    print("Print selected tool name: ", tool, "/n")

    if tool == "RAG_LLM":
        result = model.invoke(tool_input)
    elif tool =="EMAIL":
        result = plugin_instance.email_function()
    elif tool =="COMPILE":
        result = plugin_instance.compile_paper()
    elif tool =="CIRCULATE":
        result = plugin_instance.circulate_paper()
    else:
        raise ValueError
    _results[step_name] = str(result)
    return {"results": _results}

## [3] Solver 

The solver receives the full plan and generates the final response based on the responses of the tool calls from the worker.

Solver has its own prompt.

In [44]:
solve_prompt = """Solve the following task or problem. To solve the problem, we have made step-by-step Plan and \
retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might \
contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. Respond with the answer
directly with no extra words.

Task: {task}
Response:"""

def solve(state: ReWOO):
    plan = ""
    for _plan, step_name, tool, tool_input in state["steps"]:
        _results = state["results"] or {}
        for k, v in _results.items():
            tool_input = tool_input.replace(k, v)
            step_name = step_name.replace(k, v)
        plan += f"Plan: {_plan}\n{step_name} = {tool}[{tool_input}]"
    prompt = solve_prompt.format(plan=plan, task=state["task"])
    result = model.invoke(prompt)                                      # should I use LLM or RAG + LLM ???? Try just LLM first, will be faster
    return {"result": result.content}

## [4] Define Graph

Our graph defines the workflow. 

Each of the planner, worker(tool executor), and solver modules are added as nodes.

In [45]:
def _route(state):
    _step = _get_current_task(state)
    if _step is None:
        # We have executed all tasks
        return "solve"
    else:
        # We are still executing tasks, loop back to the "tool" node
        return "tool"

In [46]:
from langgraph.graph import StateGraph, END

graph = StateGraph(ReWOO)
graph.add_node("plan", get_plan)
graph.add_node("tool", tool_execution)
graph.add_node("solve", solve)
graph.add_edge("plan", "tool")
graph.add_edge("solve", END)
graph.add_conditional_edges("tool", _route)
graph.set_entry_point("plan")

app = graph.compile()

In [27]:
task

'how do I create a board paper?'

In [47]:
for s in app.stream({"task": task}):
    print(s)
    print("---")

{'plan': {'steps': [('Understand what a board paper is. ', '#E1', 'RAG_LLM', 'What is a board paper?'), ('Identify the key components required in a board paper. ', '#E2', 'RAG_LLM', 'What are the key components of a board paper?'), ('Determine who are the necessary people involved in the creation of the board paper. ', '#E3', 'RAG_LLM', 'Who are involved in the creation of a board paper?'), ('Send an email to the board members or NEC to write their parts of the board paper, based on the key components identified. ', '#E4', 'EMAIL', '#E2 and #E3'), ('Once the board members or NEC have written their parts, compile all the separate parts into one board paper. ', '#E5', 'COMPILE', '#E4'), ('After the paper is compiled, circulate the board paper to the board members for feedback and fixing if required. ', '#E6', 'CIRCULATE', '#E5'), ('Once the board paper is finalized, distribute the paper to the board members. ', '#E7', 'EMAIL', 'Finalized board paper to #E3')], 'plan_string': 'Plan: Under